In [ ]:
###### From https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
# -*- coding: utf-8 -*-
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# E is number of epochs
# N is number of samples; B is batch size
# D_in is input dimension; H is hidden dimension; D_out is output dimension.
E, N, B, D_in, H, D_out = 5, 64, 8, 1000, 100, 10

## Use GPU
dtype = torch.float
dev = torch.device(
    "cuda:0") if torch.cuda.is_available() else torch.device("cpu")

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in, device=dev, dtype=dtype)
y = torch.randn(N, D_out, device=dev, dtype=dtype)

train_ds = TensorDataset(x, y)
train_dl = DataLoader(train_ds, batch_size=B)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to(dev)

loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, )
for epoch in range(E):
    for xb, yb in train_dl:
        # Forward pass: compute predicted y by passing x to the model.
        y_pred = model(xb)

        # Compute and print loss.
        loss = loss_fn(y_pred, yb)

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()